# N-Rainhas
## importações necessárias

In [1]:
import numpy as np

## Aqui é criado uma função para gerar o estado inicial do jogo
essa função cria uma matriz numpy de dimensão NxN e atribui as N rainhas nas N colunas de forma aleatória 

In [2]:
def gerarEstadoInicial(numeroDeRainhas):
    '''
    entradas:
        * numeroDeRainhas = número de rainhas no tabuleiro
        
    saída: ndarray numpy de tamanho NxN onde N é igual
           ao número de rainhas
    '''
    
    tabuleiro = np.zeros((numeroDeRainhas, numeroDeRainhas), 
                         dtype=np.int)
    
    for i in range(numeroDeRainhas):
        posicaoAleatoria = np.random.randint(numeroDeRainhas)
        tabuleiro[posicaoAleatoria, i] = 1
        
    return tabuleiro

## Acessando as diagonais
foi necessário criar a função abaixo para que fosse possível acessar as diagonais da matriz como vetores

In [3]:
def pegarLinhaColunaDaDiagonal(dimensao, valor, tipo='prim'):
    '''
    entradas:
        * dimensao = dimensão do tabuleiro NxN
        * valor = valor calculado para atingir as diagonais
        * tipo = 'prim' para selecionar a diagonal primária e 
                 'sec' para secundária
                 
    saída: linhas e respectivas colunas para se percorrer as
           diagonais
    '''
    
    linhas = []; colunas = []
    sinal  = 1 if tipo == 'prim' else -1
    
    for i in range(dimensao):
        for j in range(dimensao):
            if i + j*sinal == valor:
                linhas.append(i)
                colunas.append(j)
                
    return linhas, colunas

## Ataque em uma linha
com essa função, é possível calcular o número total de ataques realizados ao longo de todas as linhas do tabuleiro

In [4]:
def ataquesPorLinha(tabuleiro, dimensao, ataques=0):
    '''
    entradas:
        * tabuleiro = tabuleiro do jogo
        * dimensao = dimensão do tabuleiro NxN
        * ataques = quantidade de ataques calculados
                 
    saída: número de ataques por linha
    '''
    
    for linha in range(dimensao):
        numeroDeRainhas = np.count_nonzero(tabuleiro[linha, :])
        ataques += numeroDeRainhas - 1 if numeroDeRainhas > 1 else 0
    
    return ataques

## Ataque pela diagonal primária
ao perceber que os valores de linha e colunas somados, resultavam em **valores** constantes pra um determinada diagonal primária, compreendidos de 1 a 2N-3

In [5]:
def ataquesPorDiagonalPrimaria(tabuleiro, dimensao, ataques=0):
    '''
    entradas:
        * tabuleiro = tabuleiro do jogo
        * dimensao = dimensão do tabuleiro NxN
        * ataques = quantidade de ataques calculados
                 
    saída: número de ataques por diagonal primária
    '''
    
    for valorDiagonal in range(1, 2*dimensao-3):
        linhas, colunas = pegarLinhaColunaDaDiagonal(dimensao, valorDiagonal)
        numeroDeRainhas = np.count_nonzero(tabuleiro[linhas, colunas])
        ataques += numeroDeRainhas - 1 if numeroDeRainhas > 1 else 0
        
    return ataques

## Ataque pela diagonal secundária
ao perceber que os valores de linha subtraídos pelas colunas, resultavam em **valores** constantes pra um determinada diagonal secundária, compreendidos de -N+2 à N-1

In [6]:
def ataquesPorDiagonalSecundaria(tabuleiro, dimensao, ataques=0):
    '''
    entradas:
        * tabuleiro = tabuleiro do jogo
        * dimensao = dimensão do tabuleiro NxN
        * ataques = quantidade de ataques calculados
                 
    saída: número de ataques por diagonal secundária
    '''

    for valorDiagonal in range(-dimensao+2, dimensao-1):
        linhas, colunas = pegarLinhaColunaDaDiagonal(dimensao, valorDiagonal, tipo='sec')
        numeroDeRainhas = np.count_nonzero(tabuleiro[linhas, colunas])
        ataques += numeroDeRainhas - 1 if numeroDeRainhas > 1 else 0
        
    return ataques

# Calculando o custo total
com as três funções acima combinadas, foi criado uma função para calcular o custo total, isto é, o número total de páres de damas em ataque, no que diz respeito às linhas, diagonais primárias e secundárias

In [7]:
def calcularCusto(tabuleiro):
    '''
    entrada:
        * tabuleiro = tabuleiro do jogo
                 
    saída: número de ataques no tabuleiro
    '''
    
    dimensao = tabuleiro.shape[0]
    
    ataques  = ataquesPorLinha(tabuleiro, dimensao)
    ataques += ataquesPorDiagonalPrimaria(tabuleiro, dimensao)
    ataques += ataquesPorDiagonalSecundaria(tabuleiro, dimensao)
     
    return ataques

## Gerando matriz de custos
com a função custo total criada, foi posível criar uma matriz de custo, isto é, calcular todos os custos para todos os movimentos possíveis de rainhas em suas respectivas colunas

In [8]:
def gerarMatrizDeCusto(tabuleiro):
    '''
    entrada:
        * tabuleiro = tabuleiro do jogo
                 
    saída: matriz de custo por movimento
    '''

    dimensao = tabuleiro.shape[0]
    custos   = []

    for coluna in range(dimensao):
        posicaoOriginalDaRainha = tabuleiro[:, coluna].argmax()
        tabuleiro[:, coluna] = 0
        
        for linha in range(dimensao):
            tabuleiro[linha, coluna] = 1
            custos.append(calcularCusto(tabuleiro))
            tabuleiro[linha, coluna] = 0
            
        tabuleiro[posicaoOriginalDaRainha, coluna] = 1
        
    return np.array(custos).reshape((dimensao, dimensao)).T

# Mudar para melhor vizinho
a função abaixo muda uma rainha de lugar no intuito de diminuir o número total de ataques no tabuleiro

In [9]:
def gerarPosicaoAtualRainhas(tabuleiro):
    '''
    entrada:
        * tabuleiro = tabuleiro corrente
                 
    saída: conjunto de posições das rainhas no tabuleiro
    '''
    
    linhas, colunas = np.where(tabuleiro == 1)
    return set(zip(linhas, colunas))

In [10]:
def gerarConjuntoAleatorio(tabuleiro):
    '''
    entrada:
        * tabuleiro = tabuleiro corrente
                 
    saída: conjunto de posições sem rainhas no tabuleiro
    '''
    linhas, colunas = np.where(tabuleiro == 0)
    return list(zip(linhas, colunas))

In [11]:
def gerarConjuntoDeMovimentos(matrizDeCusto):
    '''
    entrada:
        * matrizDeCusto = matriz de custo 
                 
    saída: conjunto de possíveis movimentos no tabuleiro
    '''
    
    linhas, colunas = np.where(matrizDeCusto == np.min(matrizDeCusto))
    return set(zip(linhas, colunas))

In [12]:
def mudarDeEstado(tabuleiro):
    '''
    entrada:
        * tabuleiro = tabuleiro do jogo
                 
    saída: novo estado do tabuleiro
    '''

    dimensao = tabuleiro.shape[0]
    
    posicaoRainhas = gerarPosicaoAtualRainhas(tabuleiro)
    matrizDeCusto  = gerarMatrizDeCusto(tabuleiro)
    possibilidades = gerarConjuntoDeMovimentos(matrizDeCusto)
    
    if (possibilidades - posicaoRainhas) == set():
        print('\nMínimo Local!\n')
        movimentos = gerarConjuntoAleatorio(tabuleiro)
        
    else:
        movimentos = list(possibilidades - posicaoRainhas)
        
    aleatorio  = np.random.randint(len(movimentos))
    movimento  = movimentos[aleatorio]

    tabuleiro[:, movimento[1]] = 0
    tabuleiro[movimento]       = 1
    
    print('\nMatriz de Custo') 
    print(matrizDeCusto.view())
    
    return tabuleiro

## Hill Climbing
implementação do algoritmo

In [13]:
def hillClimbing(numeroDeRainhas):
    '''
    entrada:
        * numeroDeRainhas = número de rainhas no tabuleiro
                 
    saída: tabuleiro sem rainhas em ataque
    '''
    
    tabuleiro = gerarEstadoInicial(numeroDeRainhas)
    
    print('\nEstado Inicial') 
    print(tabuleiro.view())
    
    movimentos = 0
    while(calcularCusto(tabuleiro)):
        tabuleiro = mudarDeEstado(tabuleiro)
        movimentos += 1
        
        print('\nMelhor Vizinho # %d' % movimentos) 
        print(tabuleiro.view())
        
        
    print('\nCusto:', calcularCusto(tabuleiro))
    print('Movimentos Realizados: %d\n' % movimentos)

In [23]:
%time hillClimbing(numeroDeRainhas=8)


Estado Inicial
[[0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0]]

Matriz de Custo
[[8 7 6 7 7 7 7 6]
 [8 7 7 6 8 6 9 5]
 [7 7 7 8 7 7 7 7]
 [9 6 8 7 7 6 8 5]
 [6 7 5 7 7 7 6 4]
 [7 6 8 6 9 7 8 6]
 [7 8 6 7 7 7 7 6]
 [7 5 7 5 7 6 7 5]]

Melhor Vizinho # 1
[[0 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0]]

Matriz de Custo
[[5 5 3 6 4 4 4 6]
 [5 5 4 4 6 4 6 5]
 [4 4 4 6 4 6 4 7]
 [6 4 5 5 4 4 6 5]
 [4 6 3 6 5 6 4 4]
 [4 4 5 4 6 5 6 6]
 [4 6 3 4 4 6 4 6]
 [4 3 4 3 5 4 4 5]]

Melhor Vizinho # 2
[[0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 1]
 [1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 1 0]
 [0 0 0 0 0 0 0 0]]

Matriz de Custo
[[4 5 3 5 4 3 4 4]
 [3 4 4 2 5 3 4 2]
 [4 3 4 5 5 5 3 5]
 [5 5 5 5 3 3 5 3]
 [3 6 3 5 5 5 3 3]
 [3 5 5 4 6 4 5 4]
 [4 6 3 3 5 5 